# LLMs, Prompt Engineering, and OLMo

## Introduction

An introduction to large language models and how they're trained is out of scope, but they have been trained over large amounts of textual information available on the Internet, including books, articles, websites, and other digital content. Getting into the weeds of how these models are trained is out of the scope of this tutorial, but we have added links to papers and tutorials if you'd like to understand how LLMs are trained. Do note that training LLMs is expensive; the cost can easily increase to millions. 

Early language models could predict the probability of a single word token or n-grams; modern large language models can predict the likelihood of sentences, paragraphs, or entire documents.

However, LLMs are notoriously unable to retrieve and manipulate the knowledge they possess, which leads to issues like hallucination (i.e., generating factually incorrect information), knowledge cutoffs, and poor performance in domain-specific applications.

For this entire tutorial, we will be using [Open Language Model: OLMo](https://allenai.org/olmo), an open LLM framework built by [Allen Institute for AI](https://allenai.org/). With this open framework, you can access its complete pretraining data ([dolma](https://github.com/allenai/dolma)), training code, model weights, and evaluation suite. Tracking openness, transparency, accountability, and risks in LLMs is a growing research area. Checkout this [tool](https://opening-up-chatgpt.github.io/) to understand the range of openness in these models.

We have chosen a 7B instruction-tuned OLMo model that we have compressed to speed up its inference time. 

In [1]:
from llama_cpp import Llama # Python bindings for llama.cpp, to enable LLM inference with minimal setup
from ssec_tutorials.scipy_conf import * # Contains helper methods for tutorial

In [2]:
from inspect import signature 

In [3]:
# Loads the model from huggingface hub: https://huggingface.co/ssec-uw/OLMo-7B-Instruct-GGUF
olmo = Llama.from_pretrained(repo_id="ssec-uw/OLMo-7B-Instruct-GGUF", filename="OLMo-7B-Instruct-Q4_K_M.gguf", verbose=False)

Note the `7B,` `Instruct,` `GGUF,` and `Q4_K_M` keywords here.

**7B**: B stands for billion, and 7B suggests that this specific model has 7 billion parameters

Base models, for example [AllenAi's OLMo-7B](https://huggingface.co/allenai/OLMo-7B), [AllenAi's OLMo-1B](https://huggingface.co/allenai/OLMo-1B), and [Meta's Llama-3-8B](meta-llama/Meta-Llama-3-8B) processes billions of words and texts. The training process is semi-supervised, meaning data is supplied without much annotation or labeling, but much effort is poured into improving the data quality. We have found that training the model with tremendous amount of text allows it to learn language patterns and general knowledge.

When prompted, the model predicts the next tokens (words) statistically likely to follow.

For example,

In [4]:
model_response = olmo(prompt="Jupiter is the largest", echo=True, max_tokens=1, temperature=0.8)

In [6]:
print(parse_text_generation_response(model_response))

Jupiter is the largest planet


But when prompted with, `What is the capital of Washington state in the USA?`, a base model **could** generate logical text that may or may not contain the right answer. This is when `Instruction` fine-tuning comes into play, which enhances the base model's ability to execute specific tasks. 


> Add more information about how instruct models are trained and chat templates.

In [7]:
model_response = olmo(prompt="What is the capital of Washington state in the USA?", echo=True, temperature=0.8)

In [8]:
print(parse_text_generation_response(model_response))

What is the capital of Washington state in the USA?
Olympia is the capital of the U.S. state of Washington.


In [9]:
signature(olmo).parameters

mappingproxy({'prompt': <Parameter "prompt: 'str'">,
              'suffix': <Parameter "suffix: 'Optional[str]' = None">,
              'max_tokens': <Parameter "max_tokens: 'Optional[int]' = 16">,
              'temperature': <Parameter "temperature: 'float' = 0.8">,
              'top_p': <Parameter "top_p: 'float' = 0.95">,
              'min_p': <Parameter "min_p: 'float' = 0.05">,
              'typical_p': <Parameter "typical_p: 'float' = 1.0">,
              'logprobs': <Parameter "logprobs: 'Optional[int]' = None">,
              'echo': <Parameter "echo: 'bool' = False">,
              'stop': <Parameter "stop: 'Optional[Union[str, List[str]]]' = []">,
              'frequency_penalty': <Parameter "frequency_penalty: 'float' = 0.0">,
              'presence_penalty': <Parameter "presence_penalty: 'float' = 0.0">,
              'repeat_penalty': <Parameter "repeat_penalty: 'float' = 1.1">,
              'top_k': <Parameter "top_k: 'int' = 40">,
              'stream': <Paramet

In [10]:
chat_response = olmo.create_chat_completion(
    messages=[
        {
            "role": "system",
            "content": "You are an astro physics expert that answers questions around astrophysics.",
        },
        {"role": "user", "content": "What is dark matter?"},
    ],
    temperature=0.8
)

In [16]:
signature(olmo.create_chat_completion).parameters

mappingproxy({'messages': <Parameter "messages: 'List[ChatCompletionRequestMessage]'">,
              'functions': <Parameter "functions: 'Optional[List[ChatCompletionFunction]]' = None">,
              'function_call': <Parameter "function_call: 'Optional[ChatCompletionRequestFunctionCall]' = None">,
              'tools': <Parameter "tools: 'Optional[List[ChatCompletionTool]]' = None">,
              'tool_choice': <Parameter "tool_choice: 'Optional[ChatCompletionToolChoiceOption]' = None">,
              'temperature': <Parameter "temperature: 'float' = 0.2">,
              'top_p': <Parameter "top_p: 'float' = 0.95">,
              'top_k': <Parameter "top_k: 'int' = 40">,
              'min_p': <Parameter "min_p: 'float' = 0.05">,
              'typical_p': <Parameter "typical_p: 'float' = 1.0">,
              'stream': <Parameter "stream: 'bool' = False">,
              'stop': <Parameter "stop: 'Optional[Union[str, List[str]]]' = []">,
              'seed': <Parameter "seed: 'Op

**References**
1. https://news.ycombinator.com/item?id=35712334
2. https://benjaminwarner.dev/2023/07/01/attention-mechanism
3. [Opening up ChatGPT: Tracking openness, transparency, and accountability in instruction-tuned text generators](https://dl.acm.org/doi/10.1145/3571884.3604316)
4. 